In [1]:
import torch
from pytorch_pretrained_bert import *
import logging
logging.basicConfig(level=logging.INFO)
import pandas as pd
import numpy as np
import shutil
import pickle
import tqdm
from tqdm import tqdm
import os,sys
from multiprocessing import Pool, Manager
import re
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import gensim,re
import string
import bert
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

INFO:gensim.summarization.textcleaner:'pattern' package not found; tag filters are not available for English
Using TensorFlow backend.


## Load Data
* Get Train Data
* Get Label

- Model: bert-base-multilingual-uncased

In [2]:
out_dir = './data/bert/profile_input/' #complet_input=>A  #profile_input=>B
# input_dir_complete = './data/aries_complete/' #complete=>A
# input_dir_profile_fam = './data/tree-rep-logs-profile/' #profile_fam=>B
root_dir = './data/tree-rep-logs-profile/'
fam_dir = next(os.walk(root_dir))[1]
dil= r"[,.;\-+^()/@#?!&$:{}\\*%~\'\"=_]+\ *" #等號、底線被保留? #r"[@#]+\ *" 

* 濾掉#、@
* 濾掉PR、Ret、SUBK

In [4]:
def compare_list(inputs):
    '''
        Input: two lists of string(s)
        Output: byte string
    '''
    target,srclist = inputs
    bytestr=[]
    for element in target:
        if element in srclist:
            bytestr.append(['1']*len(element.split()))
        else:
            bytestr.append(['0']*len(element.split()))
    return bytestr

In [5]:
for fam in tqdm(fam_dir):
    tree_dir = next(os.walk(root_dir + fam))[1]
    for tree in tree_dir:
        in_directory = root_dir + fam +  '/' + tree + '/'
        out_directory = out_dir + fam + '/' + tree + '/'
        if not os.path.exists(out_directory):
            os.makedirs(out_directory)
        hl_list = next(os.walk(in_directory))[2] # get all filenames in the in_directory
        hl_list = [os.path.join(in_directory, f) for f in hl_list] # filepathname list
        hl_list = list(filter(lambda f: f.endswith(".hooklog"), hl_list))
        try:
            rep_param = pickle.load(open(in_directory+'parameter_rep.pickle','rb'))
            rep = []
            for line in rep_param:
                temp = re.sub(dil," ",line)
                temp = temp.replace('PR','')
                temp = temp.replace('Ret','')
                temp = temp.replace('SUBK','')
                temp = temp.replace('<BOS>','')
                temp = temp.replace('<MOS>','')
                temp = temp.replace('<EOS>','')
                temp = temp.replace('<PAD>','')
                temp = temp.replace('<UNK>','')
                temp = temp.split(" ")
                temp = list(filter(None, temp))
                temp = ' '.join(temp)
                if temp != '':
                    rep.append(temp)
        except EOFError:
            rep=['']
#         rep = temp
        tree_hkl = []
        p = Pool(processes=8)
        for hkl in hl_list:
#             param = []
            param = []
            with open(hkl , encoding='ISO 8859-1') as f:
                profile = f.read().splitlines()
                for line in profile:
                    temp = re.sub(dil," ",line)
                    temp = temp.replace('PR','')
                    temp = temp.replace('Ret','')
                    temp = temp.replace('SUBK','')
                    temp = temp.split(" ")
                    temp = list(filter(None, temp))
                    temp = ' '.join(temp) #str: 一句話，一個row
                    param.append(temp) #list: 一個hkl全部的api，一個file
#             f.close()
            tree_hkl.append((param,rep))
        bytestr = p.map(compare_list,tree_hkl) #shape=(tree的member數,hkl長度也就是行數,單一api call長度)
        p.close()
        p.terminate()
#         bytestr = compare_list(param,rep)
#                 hkl = temp

100%|██████████| 80/80 [04:06<00:00,  4.85s/it]


In [25]:
tree_hkl

[(['RegQueryValue HKLM sys setup systemsetupinprogress REG DWORD 0 0',
   'LoadLibrary ARB windowsshell MANIFEST N',
   'CreateFile ARB MANIFEST GENERIC READ OPEN EXISTING FILE SHARE DELETE FILE SHARE READ P',
   'RegQueryValue HKCU desktop smoothscroll 0 77462a48 P',
   'RegQueryValue HKCU soft ms win explorer advanced enableballoontips 0 77462170 P',
   'RegEnumValue HKLM soft ms winNT languagepack surr REG DWORD 2 0',
   'RegQueryValue HKLM sys curCtlSet ctl sessionManager criticalsectiontimeout 0 12f9b0 0',
   'RegQueryValue HKLM soft ms ole rwlockresourcetimeout 0 12f9b4 P',
   'LoadLibrary SYS uxtheme DLL P',
   'RegQueryValue HKCU soft ms win thememanager compositing 12e7f4 12e804 P',
   'RegQueryValue HKCU desktop lamebuttontext 12ea00 5ada1620 P',
   'LoadLibrary SYS uxtheme DLL P',
   'LoadLibrary SYS shfolder DLL P',
   'LoadLibrary SYS ole32 DLL P',
   'RegQueryValue HKCU soft ms win currentversion policies explorer nonethood 12ef04 12f1b8 P',
   'RegQueryValue HKCU soft ms

In [16]:
len(bytestr[1]) , len(param) , len(hl_list[1])

(158, 158, 128)

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-uncased-vocab.txt from cache at /home/leoqaz12/.pytorch_pretrained_bert/bb773818882b0524dc53a1b31a2cc95bc489f000e7e19773ba07846011a6c711.535306b226c42cebebbc0dabc83b92ab11260e9919e21e2ab0beb301f267b4c7


In [7]:
tokenizer.tokenize(profile[0])

['reg',
 '##quer',
 '##yva',
 '##lue',
 '#',
 'pr',
 '@',
 'hk',
 '##lm',
 '@',
 'sy',
 '##s',
 '_',
 'set',
 '##up',
 '\\',
 '#',
 'pr',
 '@',
 'sub',
 '##k',
 '@',
 'systems',
 '##etu',
 '##pin',
 '##pro',
 '##gre',
 '##ss',
 '#',
 'pr',
 '@',
 'reg',
 '_',
 'dwor',
 '##d',
 '#',
 'pr',
 '@',
 '0',
 '#',
 'ret',
 '#',
 '0']

In [27]:
li = ['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']
li2 = 'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'.split()
# print(len(li),len(li2))
li2[-1]

'.'

In [14]:
hkl

'CreateFile USR DLL GENERIC_WRITE CREATE_NEW FILE_SHARE_READ P'

In [13]:
rep

['RegQueryValue HKLM sys setup systemsetupinprogress REG DWORD 0 0',
 'LoadLibrary ARB windowsshell MANIFEST N',
 'CreateFile ARB MANIFEST GENERIC READ OPEN EXISTING FILE SHARE DELETE FILE SHARE READ P',
 'RegQueryValue HKCU desktop smoothscroll 0 77462a48 P',
 'RegQueryValue HKCU soft ms win explorer advanced enableballoontips 0 77462170 P',
 'RegEnumValue HKLM soft ms winNT languagepack surr REG DWORD 2 0',
 'RegQueryValue HKLM sys curCtlSet ctl sessionManager criticalsectiontimeout 0 12f9b0 0',
 'RegQueryValue HKLM soft ms ole rwlockresourcetimeout 0 12f9b4 P',
 'LoadLibrary SYS uxtheme DLL P',
 'RegQueryValue HKCU soft ms win thememanager compositing 12e7f4 12e804 P',
 'RegQueryValue HKCU desktop lamebuttontext 12ea00 5ada1620 P',
 'LoadLibrary SYS uxtheme DLL P',
 'LoadLibrary SYS shfolder DLL P',
 'LoadLibrary SYS ole32 DLL P',
 'RegQueryValue HKCU soft ms win currentversion policies explorer nonethood 12ef04 12f1b8 P',
 'RegQueryValue HKCU soft ms win currentversion policies exp

In [14]:
param

['RegQueryValue HKLM sys setup systemsetupinprogress REG DWORD 0 0',
 'LoadLibrary ARB windowsshell MANIFEST N',
 'CreateFile ARB MANIFEST GENERIC READ OPEN EXISTING FILE SHARE DELETE FILE SHARE READ P',
 'RegQueryValue HKCU desktop smoothscroll 0 77462a48 P',
 'RegQueryValue HKCU soft ms win explorer advanced enableballoontips 0 77462170 P',
 'RegEnumValue HKLM soft ms winNT languagepack surr REG DWORD 2 0',
 'RegQueryValue HKLM sys curCtlSet ctl sessionManager criticalsectiontimeout 0 12f9b0 0',
 'RegQueryValue HKLM soft ms ole rwlockresourcetimeout 0 12f9b4 P',
 'LoadLibrary SYS uxtheme DLL P',
 'RegQueryValue HKCU soft ms win thememanager compositing 12e7f4 12e804 P',
 'RegQueryValue HKCU desktop lamebuttontext 12ea00 5ada1620 P',
 'LoadLibrary SYS uxtheme DLL P',
 'LoadLibrary SYS shfolder DLL P',
 'LoadLibrary SYS ole32 DLL P',
 'RegQueryValue HKCU soft ms win currentversion policies explorer nonethood 12ef04 12f1b8 P',
 'RegQueryValue HKCU soft ms win currentversion policies exp

In [15]:
'a' in ['a','b','c']

True

In [24]:
li = []
li.append((1,2))
li

[(1, 2)]

## Deprecated

In [5]:
out_dir = './data/bert/complete_input/' #complet_input=>A  #profile_input=>B
input_dir_complete = './data/aries_complete/' #complete=>A
# input_dir_profile_fam = './data/tree-rep-logs-profile/' #profile_fam=>B
root_dir = './data/tree-rep-logs-profile/'
fam_dir = next(os.walk(root_dir))[1]

In [14]:
for fam in tqdm(fam_dir):
    tree_dir = next(os.walk(root_dir + fam))[1]
    for tree in tree_dir:
        in_directory = root_dir + fam +  '/' + tree + '/'
        out_directory = out_dir + fam + '/' + tree + '/'
        if not os.path.exists(out_directory):
            os.makedirs(out_directory)
        hl_list = next(os.walk(in_directory))[2] # get all filenames in the in_directory
        hl_list = [os.path.join(input_dir_complete, f) for f in hl_list] # filepathname list
        hl_list = list(filter(lambda f: f.endswith(".hooklog"), hl_list))

        for hkl in hl_list:
            param = []
            with open(hkl , encoding='ISO 8859-1') as f:
                profile = f.read().splitlines()
            print(profile)
            
            rows = preprocess1(profile)
            break
        break
            
            
            
            
#             if (len(profile)<lower_bound or len(profile)>upper_bound):
#                 continue
#             else:

# with open(,encoding='ISO 8859-1')

100%|██████████| 80/80 [00:00<00:00, 564.57it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




In [29]:
'[CLS]' + "kk ii"

'[CLS]kk ii'

In [32]:
def preprocess1(my_list):
    '''
    input: complete trace (list by readlines)
    do: remove time tick, add [CLS], add[SEP]
    output: (string) all rows concatenate
    '''
    my_list = my_list[1:] #remove header file name
    indices = [i for i, x in enumerate(my_list) if '#' in x]
    # indices 
    rows = ""
    for i in range(len(indices)):
        try:
            row = " ".join(my_list[indices[i]+1:indices[i+1]])
        except IndexError:
            break
        if i == 0:
            row = '[CLS] '+ row + ' [SEP]'
        else:
            row = row + ' [SEP] '
        rows += row
    return rows
#     print(" ".join(my_list[indices[i]+1:indices[i+1]]))
# print(rows)

In [19]:
for element in profile[1:]:
    

['#313430000',
 'RegOpenKey',
 'hKey=HKEY_LOCAL_MACHINE',
 'samDesired=20019',
 'phkresult=12f7a4',
 'ulOptions=0',
 'lpSubKey=Software\\Microsoft\\Windows NT\\CurrentVersion\\Diagnostics',
 'EAX=2',
 'Return=2',
 '#313580000',
 'LoadLibrary',
 'lpFileName=C:\\WINDOWS\\system32\\IMM32.DLL',
 'dwFlags=0',
 'EAX=76390000',
 'Return=SUCCESS',
 '#315290000',
 'LoadLibrary',
 'lpFileName=gdi32.dll',
 'dwFlags=0',
 'EAX=77f10000',
 'Return=SUCCESS',
 '#315280000',
 'LoadLibrary',
 'lpFileName=gdi32.dll',
 'dwFlags=0',
 'EAX=77f10000',
 'Return=SUCCESS',
 '#314580000',
 'LoadLibrary',
 'lpFileName=LPK.DLL',
 'dwFlags=0',
 'EAX=629c0000',
 'Return=SUCCESS',
 '#316400000',
 'RegOpenKey',
 'hKey=HKEY_LOCAL_MACHINE',
 'samDesired=2000000',
 'phkresult=12f8ac',
 'ulOptions=0',
 'lpSubKey=Software\\Microsoft\\Windows\\CurrentVersion\\Explorer\\Performance',
 'EAX=2',
 'Return=2',
 '#316610000',
 'RegOpenKey',
 'hKey=HKEY_LOCAL_MACHINE',
 'samDesired=1',
 'phkresult=12f848',
 'ulOptions=0',
 'lpSubK

In [7]:
hl_list

['./data/tree-rep-logs-profile/fakeav_0.8/G290/1c63fd98933747a026ff13ac3a73ea3d18c86d28cc580a1c3e933c4d1558e141_2968.trace.hooklog',
 './data/tree-rep-logs-profile/fakeav_0.8/G290/1bc21e157ecab582d9e4cb2262cb447c11187bf391b1d271dcce6ec28f14c0ef_2972.trace.hooklog']

In [34]:
# !wget https://raw.githubusercontent.com/bheinzerling/dougu/master/dougu/bert.py

--2019-04-02 08:10:52--  https://raw.githubusercontent.com/bheinzerling/dougu/master/dougu/bert.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.72.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.72.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5477 (5.3K) [text/plain]
Saving to: ‘bert.py’

bert.py             100%[===================>]   5.35K  --.-KB/s    in 0s      

2019-04-02 08:10:52 (59.8 MB/s) - ‘bert.py’ saved [5477/5477]



In [41]:
# bert = bert.Bert.Model("bert-base-cased")
# bert = dougu.bert.Bert.Model("bert-base-cased")
sentences = ['oo kk ii','mm nn bb']
featurized_sentences = []
for tokens in sentences:
    features = {}
    features["bert_ids"], features["bert_mask"], features["bert_token_starts"] = bert.subword_tokenize_to_ids(tokens)
    featurized_sentences.append(features)

In [42]:
featurized_sentences

[{'bert_ids': tensor([[101, 184, 184, 180, 180, 178, 178, 102,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0, 

In [29]:
def collate_fn(featurized_sentences_batch):
    bert_batch = [torch.cat(features[key] for features in featurized_sentences], dim=0) for key in ("bert_ids", "bert_mask", "bert_token_starts")]
    return bert_batch

SyntaxError: invalid syntax (<ipython-input-29-2f1b7721b62f>, line 2)